In [1]:
pip install opencv-python==3.4.18.65

  Using cached opencv_python-3.4.18.65-cp36-abi3-macosx_10_15_x86_64.whl.metadata (18 kB)
Using cached opencv_python-3.4.18.65-cp36-abi3-macosx_10_15_x86_64.whl (44.3 MB)
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.9.0
ERROR: Cannot uninstall opencv-python 4.9.0, RECORD file not found. Hint: The package was installed by conda.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import pyautogui
import matplotlib.pyplot as plt
import apriltag
import argparse

In [ ]:
# init part
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
pts1 = []
pts2 = []
# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()

#***DEFAULT PARAMETERS***#

# Change thresholds (0 to 255) 
params.minThreshold = 0
params.maxThreshold = 255

#Area - area in terms of pixels
params.filterByArea = True
params.minArea = 300
params.maxArea = 10000

#Circularity - measures how close the blob to a circle
params.filterByCircularity = True
params.minCircularity = 0.5 #"cicularity" of a square = 0.785 , circle = 1.000
params.maxCircularity = 1.00 

#Color - 0 (select darker) to  255 (select lighter)
params.filterByColor = False
#params.minColor = 0 
#params.maxColor = 255 

#Convexity - (Area of the Blob / Area of it’s convex hull)
params.filterByConvexity = False
params.minConvexity = 0.87
params.maxConvexity = 1.00

#Inertia - how elongated a shape is, line = 0.000, circle = 1.000,
params.filterByInertia = False
params.minInertiaRatio = 0.95
params.maxInertiaRatio = 1.00

#Repeatability
params.minRepeatability=1

font = cv2.FONT_HERSHEY_SIMPLEX

def lineLineIntersection(A, B, C, D):
    # Line AB represented as a1x + b1y = c1
    a1 = B[1] - A[1]
    b1 = A[0]- B[0]
    c1 = a1*(A[0]) + b1*(A[1])
 
    # Line CD represented as a2x + b2y = c2
    a2 = D[1] - C[1]
    b2 = C[0] - D[0]
    c2 = a2*(C[0]) + b2*(C[1])
 
    determinant = a1*b2 - a2*b1
 
    if (determinant == 0):
        # The lines are parallel. This is simplified
        # by returning a pair of FLT_MAX
        return (10**9, 10**9)
    else:
        x = int((b2*c1 - b1*c2)/determinant)
        y = int((a1*c2 - a2*c1)/determinant)
        return (x, y)

def nothing(x):
    pass

def main():
    #wait for key inputs
    #key = cv2.waitKey(1)

    #start with cursor not connected
    #cursorOn =  False
    validCenter = False
    pts1 = np.float32([[0,0],[0,0],[0,0],[0,0]])
    pts2 = np.float32([[0,0],[0,0],[0,0],[0,0]])
    warpMatrix = cv2.getPerspectiveTransform(pts1,pts2)    
    centerX = 0 
    centerY = 0


    screen_w, screen_h = pyautogui.size()
    tempCap0 = cv2.VideoCapture(0)
    tempCap1 = cv2.VideoCapture(1)
    tempCap2 = cv2.VideoCapture(2)

    print(tempCap0.getBackendName())
    eyeCap = cv2.VideoCapture(0)

    screenCap = cv2.VideoCapture(1)
    
    #pcw = 'Parameter Control Window'
    #cv2.namedWindow(pcw)

    image = 'Pupil Tracking Live Feed'
    cv2.namedWindow(image)
    cv2.resizeWindow(image,int(screen_w/2),int(screen_h/2))
    
    few = 'Fixed Eye Window'
    cv2.namedWindow(few)
    cv2.resizeWindow(few,int(screen_w/2),int(screen_h/2))
    
    slf = 'Screen Live Feed'
    cv2.namedWindow(slf)
    cv2.resizeWindow(slf,int(screen_w/2),int(screen_h/2))
    
    fsw = 'Fixed Screen Window'
    cv2.namedWindow(fsw)
    cv2.resizeWindow(fsw,int(screen_w/2),int(screen_h/2))



    
    threshold = 20
    areaParam = 1000
    circularityParam = 100
    colorParam = 20
    convexityParam = 100
    inertiaParam = 100
    
    
    while True:
        gotWarp = False
        warpMatrix = warpMatrix
        '''
        if (key == ord('c')) and not cursorOn:
            print("Cursor Connected")
            cursorOn = True
        elif (key == ord('c')) and cursorOn:
            print("Cursor Disconnected")
            cursorOn = False
        '''
            
       #frame for eye
        _, eyeFrame = eyeCap.read()
        #frame for screen
        _, screenFrame = screenCap.read()

        fixedEye = eyeFrame.copy()
        fixedScreen = screenFrame.copy()
        
        #convert to grayscale
        gray_frame = cv2.cvtColor(screenFrame, cv2.COLOR_BGR2GRAY)

        #print("[INFO] detecting AprilTags...")
        options = apriltag.DetectorOptions(families="tag36h11", quad_blur = 2.0)
        detector = apriltag.Detector(options)
        results = detector.detect(gray_frame)
        #print("[INFO] {} total AprilTags detected".format(len(results)))

        cornersList = []
        
        # loop over the AprilTag detection results
        for r in results:
            # extract the bounding box (x, y)-coordinates for the AprilTag
            # and convert each of the (x, y)-coordinate pairs to integers
            (ptA, ptB, ptC, ptD) = r.corners
            ptB = (int(ptB[0]), int(ptB[1]))
            ptC = (int(ptC[0]), int(ptC[1]))
            ptD = (int(ptD[0]), int(ptD[1]))
            ptA = (int(ptA[0]), int(ptA[1]))

            # draw the bounding box of the AprilTag detection
            cv2.line(screenFrame, ptA, ptB, (0, 255, 0), 2)
            cv2.line(screenFrame, ptB, ptC, (0, 255, 0), 2)
            cv2.line(screenFrame, ptC, ptD, (0, 255, 0), 2)
            cv2.line(screenFrame, ptD, ptA, (0, 255, 0), 2)

            # draw the center (x, y)-coordinates of the AprilTag
            (cX, cY) = (int(r.center[0]), int(r.center[1]))
            cornersList.append((cX, cY))
            cv2.circle(screenFrame, (cX, cY), 5, (0, 0, 255), -1)

            # draw the tag family on the image
            tagFamily = r.tag_family.decode("utf-8")
            cv2.putText(screenFrame, tagFamily, (ptA[0], ptA[1] - 15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            #print("[INFO] tag family: {}".format(tagFamily))

        #calculate centroid
        if(len(results) == 4):
            #draw 4 side of screen and diagonals
            #arrange corners as such
            #AB
            #CD
            print(cornersList)
            print("Sorted list")
            sorted_list =  sorted(cornersList)
            print(sorted_list)
            
            if (sorted_list[0][1] < sorted_list[1][1]):
                cornerA = sorted_list[0]
                cornerC = sorted_list[1]
            else:
                cornerA = sorted_list[1]
                cornerC = sorted_list[0]
                
            if (sorted_list[2][1] < sorted_list[3][1]):
                cornerB = sorted_list[2]
                cornerD = sorted_list[3]
            else:
                cornerB = sorted_list[3]
                cornerD = sorted_list[2]
                
            print(cornerA)
            print(cornerB)
            print(cornerC)
            print(cornerD)
            
            #print sides of "screen"
            cv2.line(screenFrame, cornerA, cornerB, (0, 255, 0), 2)
            cv2.line(screenFrame, cornerA, cornerC, (0, 255, 0), 2)
            cv2.line(screenFrame, cornerD, cornerC, (0, 255, 0), 2)
            cv2.line(screenFrame, cornerD, cornerB, (0, 255, 0), 2)   
            #print diagonals of "screen"
            cv2.line(screenFrame, cornerA, cornerD, (0, 0, 255), 2)     
            cv2.line(screenFrame, cornerC, cornerB, (0, 0, 255), 2)
            
            #get intersection of diagonals
            intersectionTuple = lineLineIntersection(cornerA, cornerD, cornerB, cornerC)
            centerX = intersectionTuple[0]
            centerY = intersectionTuple[1]
            #print center of "screen"            
            screenFrame = cv2.circle(screenFrame, intersectionTuple, 5, (0,0,255), 5)

            pts1 = np.float32([cornerA, cornerB,cornerC,cornerD])
            pts2 = np.float32([[0,0],[screen_w,0],[0,screen_h],[screen_w,screen_h]])
            warpMatrix = cv2.getPerspectiveTransform(pts1,pts2)
            fixedScreen = cv2.warpPerspective(screenFrame, warpMatrix, (int(screen_w),int(screen_h)), flags=cv2.INTER_LINEAR)
            validCenter = True
            gotWarp = True       
        elif len(results) > 1 and not (len(results) > 3):
            #warpMatrix = np.float32(results[0].homography)
            fixedScreen = cv2.warpPerspective(screenFrame, warpMatrix, (int(screen_w),int(screen_h)), flags=cv2.INTER_LINEAR)
            validCenter = False
            gotWarp = True       

        
        #Add text to frame to update displayed parameters
        eyeFrame = cv2.rectangle(eyeFrame,(0,0),(2000,60),(0,0,0),-5) #black background for white text
        eyeFrame = cv2.putText(eyeFrame, f'Min Area: {params.minArea}', (10, 15), font, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Max Area: {params.maxArea}', (10, 55), font, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Min Circ: {params.minCircularity}', (200, 15), font, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Max Circ: {params.maxCircularity}', (200, 55), font, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Color: {colorParam}', (400, 15), font, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Convexity: {convexityParam}', (600, 15), font, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Inertia: {inertiaParam}', (800, 15), font, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Threshold: {inertiaParam}', (1000, 15), font, 0.5, (255,255,255), 1)

        #Update Parameters
        params.minArea = 1000
        params.maxArea = 5000
        params.minCircularity = 0.5 #circularityParam/100
        params.maxCircularity = 1 #circularityParam/100
        #params.minColor = 0 # no minColor
        #params.maxColor = 255 # no maxColor
        params.minConvexity = 0
        params.maxConvexity = 1
        params.minInertiaRatio = 0 #inertiaParam/100 
        params.maxInertiaRatio = 1 #inertiaParam/100
        params.minThreshold = 0
        params.maxThreshold = 255

        # Create a pupil detector with the parameters
        ver = (cv2.__version__).split('.')
        if int(ver[0]) < 3 :
            detector = cv2.SimpleBlobDetector(params)
        else : 
            detector = cv2.SimpleBlobDetector_create(params)

        #warp Eyeframe
        fixedEye = cv2.warpPerspective(fixedEye, warpMatrix, (int(screen_w),int(screen_h)), flags=cv2.INTER_LINEAR)

        keypoints = detector.detect(eyeFrame)
        keypointsFixed = detector.detect(fixedEye)

        points = cv2.KeyPoint_convert(keypoints)
        fixedPoints = cv2.KeyPoint_convert(keypointsFixed)

        pointsTuple = tuple(map(tuple,points))

        

        if pointsTuple:
            xcoord = int(pointsTuple[0][0])
            ycoord = int(pointsTuple[0][1])
            #if cursorOn:
            #    pyautogui.moveTo(xcoord,ycoord)

            #eyeFrame = cv2.putText(frame, 'C', (xcoord, ycoord), font, 0.5, (0,0,255), 1)
            eyeFrame = cv2.circle(eyeFrame,(xcoord, ycoord), 5, (0,0,255), 5)

        #draw screen center if valid data
        
        if validCenter:
            screenFrame = cv2.circle(screenFrame,(centerX, centerY), 5, (0,0,255), 5)

        #draw circle around blob/s
        eyeFrame = cv2.drawKeypoints(eyeFrame, keypoints, eyeFrame, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        fixedEye = cv2.drawKeypoints(fixedEye, keypointsFixed, fixedEye, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        
        eyeFrame = cv2.resize(eyeFrame, (0, 0), fx = 0.5, fy = 0.5)
        fixedEye = cv2.resize(fixedEye, (0, 0), fx = 0.5, fy = 0.5)
        screenFrame = cv2.resize(screenFrame, (0, 0), fx = 0.5, fy = 0.5)
        fixedScreen = cv2.resize(fixedScreen, (0, 0), fx = 0.5, fy = 0.5)
        

        #fixedScreen = fixedScreen[0:int(screen_h/2),0:int(screen_w/2)]
        cv2.imshow(fsw, fixedScreen)

        cv2.imshow(few, fixedEye)

        #display eye video frame
        cv2.imshow(image, eyeFrame)
        #display screen video frame
        cv2.imshow(slf, screenFrame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    eyeCap.release()
    screenCap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

# def blob_process(img, threshold, detector):
#     gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     _, img = cv2.threshold(gray_frame, threshold, 255, cv2.THRESH_BINARY)
#     img = cv2.erode(img, None, iterations=2)
#     img = cv2.dilate(img, None, iterations=4)
#     img = cv2.medianBlur(img, 5)
#     keypoints = detector.detect(img)
#     print(keypoints)
#     return keypoints

AVFOUNDATION
[(411, 140), (422, 489), (971, 492), (981, 130)]
Sorted list
[(411, 140), (422, 489), (971, 492), (981, 130)]
(411, 140)
(981, 130)
(422, 489)
(971, 492)
[(399, 146), (413, 494), (959, 491), (965, 132)]
Sorted list
[(399, 146), (413, 494), (959, 491), (965, 132)]
(399, 146)
(965, 132)
(413, 494)
(959, 491)
[(409, 487), (395, 138), (956, 485), (963, 125)]
Sorted list
[(395, 138), (409, 487), (956, 485), (963, 125)]
(395, 138)
(963, 125)
(409, 487)
(956, 485)
[(404, 490), (390, 141), (952, 488), (958, 129)]
Sorted list
[(390, 141), (404, 490), (952, 488), (958, 129)]
(390, 141)
(958, 129)
(404, 490)
(952, 488)
[(383, 128), (401, 480), (951, 474), (956, 112)]
Sorted list
[(383, 128), (401, 480), (951, 474), (956, 112)]
(383, 128)
(956, 112)
(401, 480)
(951, 474)
[(383, 115), (400, 469), (950, 465), (959, 102)]
Sorted list
[(383, 115), (400, 469), (950, 465), (959, 102)]
(383, 115)
(959, 102)
(400, 469)
(950, 465)
[(398, 465), (380, 112), (948, 461), (956, 97)]
Sorted list
[(3

/private/var/folders/8v/p7szzjzd7bx0jz_vlzbg0m700000gn/T/pip-install-wbqxel8q/apriltag_0c40b5eef5104d16ae78977fd022c006/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(


[(398, 472), (382, 121), (946, 469), (954, 109)]
Sorted list
[(382, 121), (398, 472), (946, 469), (954, 109)]
(382, 121)
(954, 109)
(398, 472)
(946, 469)
[(380, 122), (397, 474), (945, 470), (951, 109)]
Sorted list
[(380, 122), (397, 474), (945, 470), (951, 109)]
(380, 122)
(951, 109)
(397, 474)
(945, 470)
[(381, 120), (397, 473), (946, 469), (954, 108)]
Sorted list
[(381, 120), (397, 473), (946, 469), (954, 108)]
(381, 120)
(954, 108)
(397, 473)
(946, 469)
[(382, 123), (398, 475), (946, 471), (954, 110)]
Sorted list
[(382, 123), (398, 475), (946, 471), (954, 110)]
(382, 123)
(954, 110)
(398, 475)
(946, 471)
[(381, 125), (397, 476), (945, 473), (953, 112)]
Sorted list
[(381, 125), (397, 476), (945, 473), (953, 112)]
(381, 125)
(953, 112)
(397, 476)
(945, 473)
[(383, 129), (398, 480), (946, 477), (953, 117)]
Sorted list
[(383, 129), (398, 480), (946, 477), (953, 117)]
(383, 129)
(953, 117)
(398, 480)
(946, 477)
[(384, 132), (398, 482), (946, 480), (953, 120)]
Sorted list
[(384, 132), (3

/private/var/folders/8v/p7szzjzd7bx0jz_vlzbg0m700000gn/T/pip-install-wbqxel8q/apriltag_0c40b5eef5104d16ae78977fd022c006/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(


[(488, 194), (469, 560), (1057, 607), (1090, 224)]
Sorted list
[(469, 560), (488, 194), (1057, 607), (1090, 224)]
(488, 194)
(1090, 224)
(469, 560)
(1057, 607)
[(484, 199), (466, 564), (1053, 610), (1085, 228)]
Sorted list
[(466, 564), (484, 199), (1053, 610), (1085, 228)]
(484, 199)
(1085, 228)
(466, 564)
(1053, 610)
[(485, 201), (466, 566), (1053, 612), (1085, 230)]
Sorted list
[(466, 566), (485, 201), (1053, 612), (1085, 230)]
(485, 201)
(1085, 230)
(466, 566)
(1053, 612)
[(488, 199), (469, 564), (1055, 612), (1088, 229)]
Sorted list
[(469, 564), (488, 199), (1055, 612), (1088, 229)]
(488, 199)
(1088, 229)
(469, 564)
(1055, 612)
[(468, 566), (488, 201), (1055, 614), (1087, 231)]
Sorted list
[(468, 566), (488, 201), (1055, 614), (1087, 231)]
(488, 201)
(1087, 231)
(468, 566)
(1055, 614)
[(488, 200), (469, 564), (1056, 612), (1088, 230)]
Sorted list
[(469, 564), (488, 200), (1056, 612), (1088, 230)]
(488, 200)
(1088, 230)
(469, 564)
(1056, 612)
[(488, 199), (468, 564), (1055, 612), (1

...

[(384, 347), (390, 692), (943, 682), (926, 326)]
Sorted list
[(384, 347), (390, 692), (926, 326), (943, 682)]
(384, 347)
(926, 326)
(390, 692)
(943, 682)
[(384, 345), (390, 690), (943, 680), (928, 325)]
Sorted list
[(384, 345), (390, 690), (928, 325), (943, 680)]
(384, 345)
(928, 325)
(390, 690)
(943, 680)
[(383, 344), (390, 690), (943, 679), (927, 324)]
Sorted list
[(383, 344), (390, 690), (927, 324), (943, 679)]
(383, 344)
(927, 324)
(390, 690)
(943, 679)
[(386, 348), (391, 694), (944, 684), (928, 328)]
Sorted list
[(386, 348), (391, 694), (928, 328), (944, 684)]
(386, 348)
(928, 328)
(391, 694)
(944, 684)
[(384, 349), (390, 694), (942, 684), (926, 328)]
Sorted list
[(384, 349), (390, 694), (926, 328), (942, 684)]
(384, 349)
(926, 328)
(390, 694)
(942, 684)
[(382, 347), (389, 692), (941, 682), (925, 326)]
Sorted list
[(382, 347), (389, 692), (925, 326), (941, 682)]
(382, 347)
(925, 326)
(389, 692)
(941, 682)
[(382, 348), (387, 694), (940, 683), (925, 328)]
Sorted list
[(382, 348), (3

/private/var/folders/8v/p7szzjzd7bx0jz_vlzbg0m700000gn/T/pip-install-wbqxel8q/apriltag_0c40b5eef5104d16ae78977fd022c006/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(


[(379, 348), (386, 695), (941, 682), (923, 325)]
Sorted list
[(379, 348), (386, 695), (923, 325), (941, 682)]
(379, 348)
(923, 325)
(386, 695)
(941, 682)
[(378, 348), (385, 695), (940, 682), (923, 325)]
Sorted list
[(378, 348), (385, 695), (923, 325), (940, 682)]
(378, 348)
(923, 325)
(385, 695)
(940, 682)
[(377, 349), (385, 696), (940, 683), (922, 327)]
Sorted list
[(377, 349), (385, 696), (922, 327), (940, 683)]
(377, 349)
(922, 327)
(385, 696)
(940, 683)
[(377, 347), (385, 695), (940, 682), (922, 326)]
Sorted list
[(377, 347), (385, 695), (922, 326), (940, 682)]
(377, 347)
(922, 326)
(385, 695)
(940, 682)
[(377, 347), (385, 695), (940, 682), (923, 326)]
Sorted list
[(377, 347), (385, 695), (923, 326), (940, 682)]
(377, 347)
(923, 326)
(385, 695)
(940, 682)
[(378, 346), (386, 694), (941, 681), (924, 324)]
Sorted list
[(378, 346), (386, 694), (924, 324), (941, 681)]
(378, 346)
(924, 324)
(386, 694)
(941, 681)
[(381, 348), (389, 695), (944, 683), (926, 326)]
Sorted list
[(381, 348), (3

/private/var/folders/8v/p7szzjzd7bx0jz_vlzbg0m700000gn/T/pip-install-wbqxel8q/apriltag_0c40b5eef5104d16ae78977fd022c006/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(


[(348, 108), (370, 469), (926, 452), (928, 88)]
Sorted list
[(348, 108), (370, 469), (926, 452), (928, 88)]
(348, 108)
(928, 88)
(370, 469)
(926, 452)
[(351, 110), (373, 470), (929, 455), (931, 91)]
Sorted list
[(351, 110), (373, 470), (929, 455), (931, 91)]
(351, 110)
(931, 91)
(373, 470)
(929, 455)
[(360, 101), (383, 462), (940, 447), (942, 81)]
Sorted list
[(360, 101), (383, 462), (940, 447), (942, 81)]
(360, 101)
(942, 81)
(383, 462)
(940, 447)
[(386, 461), (362, 100), (944, 444), (946, 77)]
Sorted list
[(362, 100), (386, 461), (944, 444), (946, 77)]
(362, 100)
(946, 77)
(386, 461)
(944, 444)
[(364, 101), (389, 462), (947, 445), (949, 78)]
Sorted list
[(364, 101), (389, 462), (947, 445), (949, 78)]
(364, 101)
(949, 78)
(389, 462)
(947, 445)
[(366, 108), (389, 468), (946, 452), (948, 86)]
Sorted list
[(366, 108), (389, 468), (946, 452), (948, 86)]
(366, 108)
(948, 86)
(389, 468)
(946, 452)
[(365, 107), (388, 467), (946, 450), (947, 84)]
Sorted list
[(365, 107), (388, 467), (946, 450